In [5]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
seed = 1
import numpy as np
import random
np.random.seed(seed)
random.seed(seed)
import os 
import csv
import pickle
import time
import h5py
from collections import defaultdict
import json
import matplotlib.pyplot as plt
import numpy.ma as ma
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICE'] = '4'
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
# config.gpu_options.per_process_gpu_memory_fraction = 0.2

In [22]:
from utils import load_poi2region, get_sequences, digitize_datetime, group_data_by_id, devide_train_and_test, \
    load_embeddings, load_embedding_from_text
from tqdm import tqdm
from dataloader import load_data, group_data_by_id
from parser import get_parser
from recommand import get_recommandation_result

In [7]:
args = get_parser(['--CITY', 'NYC'])
args.pattern = 'hand'
poi2region = load_poi2region(os.path.join('data', '{}_poi2region.pk'.format(args.CITY)))

In [8]:
origin_data, dicts = load_data(os.path.join(args.ROOT, 'data','{}_INTV_processed_voc5_len2_setting_WITH_GPS_WITH_TIME_WITH_USERID.pk'.format(args.CITY) ))
args.vocabulary_size = dicts.vocabulary_size
max_user_id = max(group_data_by_id(origin_data)[1])+1

In [11]:
seqs = get_sequences(origin_data)
trainset, testset = devide_train_and_test(seqs)

In [189]:
def get_GE_embeddings(args):
    def complement_embeddings(embeddings, N):
        tmp = np.zeros(shape=(N, embeddings.shape[1]-1))
        for i, emb in enumerate(embeddings):
            id = int(emb[0])
            tmp[id][:] = emb[1:]
        return tmp

    embeddings = load_embedding_from_text(os.path.join('data', 'embeddings', args.pattern, '{}_no_word'.format(args.CITY), 'net_POI_vec.txt'))[:,1:]
    region_embeddings = complement_embeddings(load_embedding_from_text(os.path.join('data', 'embeddings', args.pattern, '{}_no_word'.format(args.CITY), 'net_reg_vec.txt')),N=100)
    time_embeddings = complement_embeddings(load_embedding_from_text(os.path.join('data', 'embeddings', args.pattern, '{}_no_word'.format(args.CITY), 'net_time_vec.txt')), args.n_timeslot)
    return embeddings, region_embeddings, time_embeddings

def get_ST_embeddings(args, path=None):
    path = os.path.join(args.ROOT, 'STSkipgram', 'benchmark_{}'.format(args.CITY)) if path is None else path
    sem_embeds = load_embeddings(os.path.join(path, '{}_sem_emb.h5'.format(args.CITY)))
    embeddings = load_embeddings(os.path.join(path, '{}_embeddings.h5'.format(args.CITY)))
    time_embeddings = load_embeddings(os.path.join(path, '{}_time_embeddings.h5'.format(args.CITY)))
    sem_dim = sem_embeds.shape[1]; geo_dim = 100 - sem_dim
    geo_embeddings = embeddings[:,sem_dim:]
#     return sem_embeds, geo_embeddings, time_embeddings
    return sem_embeds, embeddings, time_embeddings

def get_SK_wot_embeddings(args, path=None):
    path = os.path.join(args.ROOT, 'SkipGram', 'benchmark_{}_wo_time'.format(args.CITY)) if path is None else path
    embeddings = load_embeddings(os.path.join(path, '{}_embeddings.h5'.format(args.CITY)))
    return embeddings

def get_SK_wt_embeddings(args, path=None):
    path = os.path.join(args.ROOT, 'SkipGram', 'benchmark_{}_with_time'.format(args.CITY)) if path is None else path
    embeddings = load_embeddings(os.path.join(path, '{}_embeddings.h5'.format(args.CITY)))
    time_embeddings = load_embeddings(os.path.join(path, '{}_time.h5'.format(args.CITY)))
    return embeddings, time_embeddings

def get_PV_embeddings(args, path=None):
    path = os.path.join(args.ROOT, 'Baseline', 'Poi2vec', 'exp', 'no_weight') if path is None else path
    embeddings = load_embeddings(os.path.join(path, '{}_embeddings.h5'.format(args.CITY)))
    return embeddings

def get_PRME_embeddings(args, path=None):
    path = os.path.join(args.ROOT, 'Baseline', 'PRME', 'embeddings', args.CITY)
    embeddings = load_prme_text_embeddings(os.path.join(path, 'POI_S_vectors.txt'), args.vocabulary_size)
    embeddings_u = load_prme_text_embeddings(os.path.join(path, 'POI_U_vectors.txt'), args.vocabulary_size)
    user_embeddings = load_prme_text_embeddings(os.path.join(path, 'user_vectors.txt'), max_user_id)
    return embeddings, embeddings_u, user_embeddings

In [190]:
a = get_recommandation_result(testset, args, poi2region, get_ST_embeddings(args), mode='STSG')


  0%|          | 1/1044 [00:00<02:14,  7.73it/s]

Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark_NYC/NYC_sem_emb.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark_NYC/NYC_embeddings.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark_NYC/NYC_time_embeddings.h5



 57%|█████▋    | 592/1044 [00:35<00:27, 16.54it/s]


100%|██████████| 1044/1044 [01:01<00:00, 16.86it/s]

[0.13157894736842105,
 0.275914362176628,
 0.37595896520963423,
 0.43581623550401427,
 0.4802408563782337]

In [171]:
weights = [[alpha*beta,alpha*(1-beta),1-alpha] for beta in [0.2,0.4,0.5,0.6,0.8,1.0] for alpha in [0.25,0.5,0.75,1]]
re_dict = {'_'.join(map(str,w)):[] for w in weights}
for w in weights:
    re_dict['_'.join(map(str,w))] = get_k_accuracy(multiprocess_recommand(5, testset, args, poi2region, get_ST_embeddings(args), mode='STSG', weight=w))

Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark_NYC/NYC_sem_emb.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark_NYC/NYC_embeddings.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark_NYC/NYC_time_embeddings.h5



 12%|█▏        | 26/209 [00:11<01:23,  2.18it/s]


 43%|████▎     | 90/209 [00:26<00:35,  3.37it/s]


 70%|██████▉   | 146/209 [00:40<00:17,  3.64it/s]


 82%|████████▏ | 171/208 [00:52<00:11,  3.26it/s]


...

 48%|████▊     | 100/208 [00:25<00:27,  3.88it/s]


 70%|███████   | 147/209 [00:39<00:16,  3.74it/s]


 83%|████████▎ | 172/208 [00:50<00:10,  3.43it/s]


 96%|█████████▌| 200/209 [00:56<00:02,  3.51it/s]

100%|██████████| 209/209 [00:58<00:00,  3.55it/s]

100%|██████████| 209/209 [00:59<00:00,  3.52it/s]

100%|██████████| 209/209 [01:01<00:00,  3.40it/s]

Job Done, used time 62.15464472770691


In [151]:
a = np.random.randn(3)

In [154]:
np.stack([a,a,a], axis=0)

array([[ 0.68483649, -0.59546033,  0.99905124],
       [ 0.68483649, -0.59546033,  0.99905124],
       [ 0.68483649, -0.59546033,  0.99905124]])

In [146]:
results['PRME2'] = multiprocess_recommand(8,testset2, args, poi2region, get_PRME_embeddings(args), mode='PRME')

loaded embeddings from /home/haibin2/data/checkins/Baseline/PRME/embeddings/NYC/POI_S_vectors.txt, shape (5453, 101)
loaded embeddings from /home/haibin2/data/checkins/Baseline/PRME/embeddings/NYC/POI_U_vectors.txt, shape (5453, 101)
loaded embeddings from /home/haibin2/data/checkins/Baseline/PRME/embeddings/NYC/user_vectors.txt, shape (1071, 101)



 27%|██▋       | 36/131 [00:09<00:25,  3.66it/s]


 30%|███       | 39/130 [00:22<00:52,  1.73it/s]


 42%|████▏     | 55/130 [00:35<00:48,  1.53it/s]


 72%|███████▏  | 94/130 [00:47<00:18,  1.98it/s]


 98%|█████████▊| 129/131 [00:59<00:00,  2.16it/s]


100%|██████████| 130/130 [00:59<00:00,  2.19it/s]

100%|██████████| 131/131 [01:00<00:00,  2.17it/s]

 98%|█████████▊| 128/130 [00:59<00:00,  2.14it/s]

100%|██████████| 131/131 [01:01<00:00,  2.15it/s]


 99%|█████████▉| 130/131 [01:04<00:00,  2.01it/s]

100%|██████████| 131/131 [01:05<00:00,  2.00it/s]

Job Done, used time 66.20477819442749


In [116]:
# results = {}
# results['GE'] = get_recommandation_result(testset, args, poi2region, get_GE_embeddings(args), mode='GE')
# results['STSG'] = get_recommandation_result(testset, args, poi2region, get_ST_embeddings(args), mode='STSG')
# results['Skipgram_wot'] = get_recommandation_result(testset, args, poi2region, get_SK_wot_embeddings(args), mode='Skipgram_wot')
# results['Skipgram_wt'] = get_recommandation_result(testset, args, poi2region, get_SK_wt_embeddings(args), mode='Skipgram_wt')
# results['PRME'] = np_recommandation_result(testset, args, poi2region, get_PRME_embeddings(args), mode='PRME')

In [321]:
sts, ste, stt = get_ST_embeddings(args)

Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark/NYC_sem_emb.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark/NYC_embeddings.h5
Reading dadta from /home/haibin2/data/checkins/STSkipgram/benchmark/NYC_time_embeddings.h5


In [446]:
class Recommander:
    def __init__(self, embeddings, time_embeddings=None, region_embeddings=None, sem_embeddings=None):
        emb = tf.constant(embeddings,dtype=tf.float32)
        time_emb = tf.constant(time_embeddings, dtype=tf.float32) if not time_embeddings is None else None 
        region_emb = tf.constant(region_embeddings, dtype=tf.float32) if not region_embeddings is None else None
        sem_emb = tf.constant(sem_embeddings, dtype=tf.float32) if not sem_embeddings is None else None
        
        self.decays = tf.placeholder(dtype=tf.float32, shape=(None))
        self.t_now = tf.placeholder(dtype=tf.int32, shape=(None))
        self.r_now = tf.placeholder(dtype=tf.int32, shape=(None))
        self.l_xs = tf.placeholder(dtype=tf.int32, shape=(None))
        
        self.user_profile = tf.reduce_sum(tf.nn.embedding_lookup(emb, self.l_xs) * tf.reshape(self.decays, (-1,1)), axis=0, keepdims=True)
        self.score = tf.matmul(self.user_profile, tf.transpose(emb))
        if not time_embeddings is None:
            self.time_emb_now = tf.nn.embedding_lookup(time_emb, self.t_now)
            if not sem_embeddings is None:
                self.score += tf.matmul(self.time_emb_now, tf.transpose(sem_emb))
            else:
                self.score += tf.matmul(self.time_emb_now, tf.transpose(emb))
        if not region_embeddings is None:
            self.r_emb_now = tf.nn.embedding_lookup(region_emb, self.r_now)
            self.score = tf.matmul(self.r_emb_now, tf.transpose(emb))
        self.score = tf.squeeze(self.score)

In [ ]:
c = tf.matmul

In [416]:
sess = tf.Session(config=config)

In [154]:
result = get_recommandation_result(testset, args, poi2region, (embeddings, region_embeddings, time_embeddings), mode='GE')

100%|██████████| 1044/1044 [02:47<00:00,  6.24it/s]

167.37496542930603


In [418]:
sess.run(model.score, {model.decays: np.ones(shape=3), model.l_xs: [1]*3, model.t_now:[1]})

array([[-0.5405644 , -0.4284531 , -0.681084  , ..., -1.6470017 ,
        -1.4667907 , -0.67495763]], dtype=float32)

In [175]:
result = get_recommandation_result(testset, args, poi2region, (ST_sem, ST_emb, ST_time), mode='ST')

100%|██████████| 1044/1044 [00:39<00:00, 26.37it/s]


In [209]:
args.use_decay = True

In [222]:
np.matmul(np.zeros((100)), np.zeros((6,100)).T)

array([0., 0., 0., 0., 0., 0.])

In [231]:
from tqdm import tqdm

In [340]:
tf.reset_default_graph()

In [247]:
result2 = get_recommandation_result(testset, args, poi2region, (SK_embed, SK_time), mode='Skipgram')



  0%|          | 0/1044 [00:00<?, ?it/s]

  0%|          | 2/1044 [00:00<00:52, 19.73it/s]

  1%|          | 6/1044 [00:00<00:44, 23.47it/s]

  1%|          | 11/1044 [00:00<00:36, 28.33it/s]

  1%|          | 13/1044 [00:00<00:45, 22.53it/s]

  2%|▏         | 18/1044 [00:00<00:38, 26.57it/s]

  2%|▏         | 21/1044 [00:00<00:39, 25.72it/s]

  2%|▏         | 25/1044 [00:00<00:37, 26.92it/s]

  3%|▎         | 33/1044 [00:01<00:32, 30.94it/s]

  4%|▍         | 40/1044 [00:01<00:29, 34.11it/s]

  4%|▍         | 45/1044 [00:01<00:28, 34.89it/s]

  5%|▍         | 50/1044 [00:01<00:28, 35.43it/s]

  5%|▌         | 56/1044 [00:01<00:26, 36.92it/s]

  6%|▌         | 61/1044 [00:01<00:26, 37.42it/s]

  6%|▋         | 66/1044 [00:02<00:30, 32.21it/s]

  7%|▋         | 70/1044 [00:02<00:31, 31.18it/s]

  7%|▋         | 76/1044 [00:02<00:30, 32.22it/s]

  8%|▊         | 80/1044 [00:03<00:36, 26.10it/s]

  8%|▊         | 83/1044 [00:03<00:36, 26.22it/s]

  8%|▊         | 86/1044 [00:03<00:37, 2

 68%|██████▊   | 710/1044 [00:29<00:14, 23.80it/s]

 68%|██████▊   | 713/1044 [00:30<00:14, 23.64it/s]

 69%|██████▉   | 718/1044 [00:30<00:13, 23.63it/s]

 69%|██████▉   | 721/1044 [00:30<00:13, 23.64it/s]

 69%|██████▉   | 724/1044 [00:30<00:13, 23.61it/s]

 70%|██████▉   | 728/1044 [00:30<00:13, 23.64it/s]

 70%|███████   | 732/1044 [00:30<00:13, 23.67it/s]

 70%|███████   | 735/1044 [00:31<00:13, 23.43it/s]

 71%|███████   | 740/1044 [00:31<00:12, 23.46it/s]

 71%|███████   | 743/1044 [00:31<00:12, 23.47it/s]

 72%|███████▏  | 747/1044 [00:31<00:12, 23.47it/s]

 72%|███████▏  | 752/1044 [00:31<00:12, 23.56it/s]

 73%|███████▎  | 758/1044 [00:32<00:12, 23.65it/s]

 73%|███████▎  | 762/1044 [00:32<00:11, 23.67it/s]

 73%|███████▎  | 766/1044 [00:32<00:11, 23.71it/s]

 74%|███████▍  | 771/1044 [00:32<00:11, 23.74it/s]

 75%|███████▍  | 778/1044 [00:32<00:11, 23.86it/s]

 75%|███████▍  | 782/1044 [00:32<00:10, 23.83it/s]

 75%|███████▌  | 786/1044 [00:33<00:10, 23.75it/s]

 76%|███████

In [253]:
poi_result = get_recommandation_result(testset, args, poi2region, (PV_embeddings), mode='POI2VEC')



  0%|          | 0/1044 [00:00<?, ?it/s]

  1%|          | 6/1044 [00:00<00:24, 42.79it/s]

  1%|          | 12/1044 [00:00<00:24, 42.35it/s]

  1%|▏         | 15/1044 [00:00<00:28, 36.58it/s]

  2%|▏         | 19/1044 [00:00<00:28, 36.17it/s]

  2%|▏         | 24/1044 [00:00<00:28, 35.59it/s]

  3%|▎         | 33/1044 [00:00<00:25, 40.34it/s]

  4%|▍         | 40/1044 [00:00<00:23, 43.51it/s]

  4%|▍         | 45/1044 [00:01<00:22, 43.71it/s]

  5%|▍         | 51/1044 [00:01<00:23, 42.44it/s]

  5%|▌         | 56/1044 [00:01<00:23, 41.36it/s]

  6%|▌         | 61/1044 [00:01<00:23, 41.73it/s]

  6%|▋         | 66/1044 [00:01<00:27, 35.44it/s]

  7%|▋         | 70/1044 [00:01<00:27, 35.59it/s]

  7%|▋         | 77/1044 [00:02<00:26, 36.38it/s]

  8%|▊         | 81/1044 [00:02<00:32, 29.97it/s]

  8%|▊         | 85/1044 [00:02<00:33, 28.85it/s]

  8%|▊         | 88/1044 [00:03<00:33, 28.31it/s]

  9%|▊         | 91/1044 [00:03<00:38, 25.08it/s]

  9%|▉         | 94/1044 [00:03<00:37, 

 68%|██████▊   | 710/1044 [00:29<00:14, 23.83it/s]

 68%|██████▊   | 712/1044 [00:29<00:13, 23.75it/s]

 68%|██████▊   | 714/1044 [00:30<00:13, 23.65it/s]

 69%|██████▉   | 718/1044 [00:30<00:13, 23.56it/s]

 69%|██████▉   | 721/1044 [00:30<00:13, 23.55it/s]

 69%|██████▉   | 723/1044 [00:30<00:13, 23.48it/s]

 69%|██████▉   | 725/1044 [00:30<00:13, 23.44it/s]

 70%|██████▉   | 728/1044 [00:31<00:13, 23.46it/s]

 70%|██████▉   | 730/1044 [00:31<00:13, 23.44it/s]

 70%|███████   | 733/1044 [00:31<00:13, 23.35it/s]

 70%|███████   | 735/1044 [00:31<00:13, 23.04it/s]

 71%|███████   | 740/1044 [00:32<00:13, 23.05it/s]

 71%|███████   | 743/1044 [00:32<00:13, 23.07it/s]

 72%|███████▏  | 748/1044 [00:32<00:12, 23.15it/s]

 72%|███████▏  | 755/1044 [00:32<00:12, 23.28it/s]

 73%|███████▎  | 761/1044 [00:32<00:12, 23.39it/s]

 73%|███████▎  | 766/1044 [00:32<00:11, 23.38it/s]

 74%|███████▍  | 770/1044 [00:32<00:11, 23.42it/s]

 74%|███████▍  | 774/1044 [00:33<00:11, 23.40it/s]

 75%|███████

In [256]:
ge_result = np.array(res_mat)
sk_result = np.array(result2)
sts_result = np.array(result)

In [155]:
res_mat = np.array(a)

In [33]:
def get_k_accuracy(res_mat):
    res_mat = np.array(res_mat)
    return [np.mean(np.sum(res_mat[:,:k], axis=1) > 0) for k in [1,5,10,15,20]]

In [178]:
get_k_accuracy(res_mat)

[0.10343443354148082,
 0.2645851917930419,
 0.36752899197145406,
 0.42649420160570917,
 0.4631578947368421]

In [179]:
get_k_accuracy(result)

[0.1328278322925959,
 0.26882247992863517,
 0.3765388046387154,
 0.43599464763603923,
 0.47756467439785905]

In [259]:
res_dicts = {key:get_k_accuracy(x) for key, x in zip(['LOC2VEC', 'STgram', 'skipgram', 'GE'], 
                                                     [poi_result, sts_result, sk_result, ge_result])}

In [135]:
acc_dicts = {key: get_k_accuracy(v) for key, v in results.items()}

In [186]:
def compare_models(res_dict, Ks=[1,5,10,15,20]):
    Ks = [1,5,10,15,20]
    keys = sorted(res_dict.keys())
    color_map = [np.random.uniform(size=(3)) for i in range(len(keys))]
    width = 0.3
    gap = 1
    position = 0
    tick_pos = []
    for j, k in enumerate(Ks):
        xs = np.linspace(position, position+len(keys)*width, len(keys))
        tick_pos.append(xs[int(len(xs)/2)])

        values = [res_dict[key][j] for key in keys]
        ptrs = plt.bar(xs, values, width)
        for i, p in enumerate(ptrs): p.set_facecolor(color_map[i])

        position += (width*len(keys)+1 + gap)
    xticks = np.linspace(0, position-1, len(Ks))
    plt.legend(ptrs, keys)
    plt.xticks(tick_pos, ['K={}'.format(k) for k in Ks])
    # plt.xticklabels()
    plt.title('recommendation')
    fig = plt.gcf()
    fig.set_size_inches(30,10)

In [169]:
acc_dicts['Best'] = \
[0.13991971454058877,
 0.28599464763603927,
 0.38813559322033897,
 0.45044603033006242,
 0.48898305084745763]

In [164]:
acc_dicts['second'] = re_dict['1.0_0.0_1']

In [198]:
# compare_models(acc_dicts)

In [110]:
from recommand_np import get_recommandation_result as np_recommandation_result

In [72]:
args.use_decay = True

In [73]:
ge_result = np_recommandation_result(testset, args, poi2region, get_GE_embeddings(args), mode='GE')

  0%|          | 2/1044 [00:00<01:28, 11.76it/s]

loaded embeddings from data/embeddings/hand/NYC_no_word/net_POI_vec.txt, shape (5453, 101)
loaded embeddings from data/embeddings/hand/NYC_no_word/net_reg_vec.txt, shape (89, 101)
loaded embeddings from data/embeddings/hand/NYC_no_word/net_time_vec.txt, shape (6, 101)


100%|██████████| 1044/1044 [04:28<00:00,  3.88it/s]


In [75]:
from multiprocess import multiprocess_recommand

In [77]:
ge_result2 = multiprocess_recommand(5, testset, args, poi2region, get_GE_embeddings(args), mode='GE')

loaded embeddings from data/embeddings/hand/NYC_no_word/net_POI_vec.txt, shape (5453, 101)
loaded embeddings from data/embeddings/hand/NYC_no_word/net_reg_vec.txt, shape (89, 101)
loaded embeddings from data/embeddings/hand/NYC_no_word/net_time_vec.txt, shape (6, 101)


100%|██████████| 209/209 [00:22<00:00,  9.18it/s]


Job Done, used time 23.378190517425537
